<a href="https://colab.research.google.com/github/Kostogot/Two/blob/main/Binan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
 import pandas as pd # for read file 
import warnings; warnings.simplefilter('ignore') #ignore warnings
pd.set_option('display.max_columns', None) # display all column
B = pd.read_excel('/binan.xlsx', 1) # read file
#B

In [7]:
P = [] # list
Psh = [] 
for n in range(0,31): # cycle for profit cells 
  B['profit'][n] = 0 # clear cell for calculation 
  B['%'][n] = 0
  Per = 0
  j = 0
  V = B['volume'][n] # read volume
  if B['operation'][n] == 'buy' or B['operation'][n] == 'buysh': # if operation buy
    B['ostatok'][n] = B['volume'][n] # the remainder does not change
    B['%'][n] = -B['fee'][n]/(V*B['price'][n])*100
    if B['operation'][n] == 'buy':
      P.append([B['price'][n],n]) # price add 
    else:
      Psh.append([B['price'][n],n])
  else:
    if B['operation'][n] == 'sell':
      P.sort() # sort of price, from min to max
      for i in range(0,len(P)): # open purchase cycle
        if B['ostatok'][P[i][1]] > 0 and V > 0: # if the purchase is not exhausted and there is something to sell 
          if V > B['ostatok'][P[i][1]]: # sell more buy volume 
            B['profit'][n] += (B['price'][n]-B['price'][P[i][1]])*B['ostatok'][P[i][1]] # calculation profit
            B['%'][n] += (B['price'][n]-B['price'][P[i][1]])/B['price'][P[i][1]] # calculate base for %
            Per += 1/(B['price'][P[i][1]]*B['ostatok'][P[i][1]]) 
            V -= B['ostatok'][P[i][1]] # reducing the remaining money
            B['ostatok'][P[i][1]] = 0
          else:
            B['profit'][n] += (B['price'][n]-B['price'][P[i][1]])*V # calculation profit
            B['%'][n] += (B['price'][n]-B['price'][P[i][1]])/B['price'][P[i][1]] # calculate base for %
            Per += 1/(B['price'][P[i][1]]*V)  
            B['ostatok'][P[i][1]] = round(B['ostatok'][P[i][1]]-V, 10) # calculate remainder from buy, round for show null without e+   
            V = 0
          j += 1         
    else:
      Psh.sort(reverse=True) # sort of price, from min to max
      for i in range(0,len(Psh)): # open purchase cycle        
        if B['ostatok'][Psh[i][1]] > 0 and V > 0: # if the purchase is not exhausted and there is something to sell 
          if V > B['ostatok'][Psh[i][1]]: # sell more buy volume 
            B['profit'][n] += (B['price'][Psh[i][1]]-B['price'][n])*B['ostatok'][Psh[i][1]] # calculation profit
            B['%'][n] += (B['price'][Psh[i][1]]-B['price'][n])/B['price'][Psh[i][1]] # calculate base for %
            Per += 1/(B['price'][Psh[i][1]]*B['ostatok'][Psh[i][1]]) 
            V -= B['ostatok'][Psh[i][1]] # reducing the remaining money
            B['ostatok'][Psh[i][1]] = 0
          else:
            B['profit'][n] += (B['price'][Psh[i][1]]-B['price'][n])*V # calculation profit
            B['%'][n] += B['price'][Psh[i][1]]*V # calculate base for %
            Per += 1/(B['price'][Psh[i][1]]*V) 
            B['ostatok'][Psh[i][1]] = round(B['ostatok'][Psh[i][1]]-V, 10) # calculate remainder from buy, round for show null without e+   
            V = 0
          j += 1
    B['%'][n] = (B['%'][n]-B['fee'][n]/j*Per)/j*100  # calculate % of profit 
  B['profit'][n] -= B['fee'][n] # subtraction of profit
#B

In [6]:
#B.loc[28:,['profit', '%', 'ostatok']]

In [5]:
B[['profit', '%', 'ostatok']].to_excel('./binan_prog.xlsx')